# 5강) BERT를 활용한 Dense Passage Retrieval 실습

### Requirements

In [1]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 245kB 29.7MB/s eta 0:00:01
     |████████████████████████████████| 122kB 39.3MB/s 
     |████████████████████████████████| 245kB 33.8MB/s 
     |████████████████████████████████| 2.5MB 32.8MB/s 
     |████████████████████████████████| 3.3MB 34.6MB/s 
     |████████████████████████████████| 901kB 33.2MB/s 
  Found existing installation: huggingface-hub 0.0.10
    Uninstalling huggingface-hub-0.0.10:
      Successfully uninstalled huggingface-hub-0.0.10


## 데이터셋 로딩


KorQuAD train 데이터셋을 search corpus 및 학습 데이터로 활용

In [2]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


In [3]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

9606

## 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, hugginface에서 제공하는 "bert-base-multilingual-cased" tokenizer를 활용

In [4]:
from transformers import AutoTokenizer
import numpy as np

model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [5]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [6]:
print(corpus[0])
tokenized_input = tokenizer(corpus[0], padding="max_length", truncation=True)
tokenizer.decode(tokenized_input['input_ids'])

2016년 이전까지만 해도 사토시 나카모토의 정체가 밝혀진 일은 없었다. 이름은 전형적인 일본식 이름이지만, 비트코인의 개발을 발표할 당시에는 사토시의 이름이 영어로만 쓰여있었고, 일본계 이름에 거의 반드시 붙어있는 한자 표기를 밝히지 않았던 관계로, 일본인으로 위장한 사람일 가능성도 제기되었다. 실제로 '사토시 나카모토'라는 이름을 본명으로 사용하는 한 일본계 미국인이 잠시 사토시의 정체라고 추정되었으나, 사토시라고 지목된 당사자는 스스로 '비트코인의 개발자 사토시'가 아니며, 동명이인임을 주장하였다. 또한 한때는 일본의 수학자로 교토 대학의 교수인 모치즈키 신이치가 사토시라고 지목되기도 했으나, 모치즈키 신이치는 자신은 사토시가 아니라고 했다. 실제로 사토시는 일본어를 쓴 적이 단 한번도 없었고, 비트코인 자체가 오픈 소스 소프트웨어 상에 구현된 전자화폐라서 실제 개발자가 누구인지 알아내기는 매우 힘드므로, 일본인이 아닐 가능성도 제기되었다. 또는 '사토시 나카모토'라는 이름 자체가 동북아시아와 북아메리카 지역의 대표적인 대기업들인 나카미치, 모토로라, 삼성, 도시바의 4개 기업이 공동으로 쓰는 가명이라는 설도 제기된 바 있다.


"[CLS] 2016년 이전까지만 해도 사토시 나카모토의 정체가 밝혀진 일은 없었다. 이름은 전형적인 일본식 이름이지만, 비트코인의 개발을 발표할 당시에는 사토시의 이름이 영어로만 쓰여있었고, 일본계 이름에 거의 반드시 붙어있는 한자 표기를 밝히지 않았던 관계로, 일본인으로 위장한 사람일 가능성도 제기되었다. 실제로'사토시 나카모토'라는 이름을 본명으로 사용하는 한 일본계 미국인이 잠시 사토시의 정체라고 추정되었으나, 사토시라고 지목된 당사자는 스스로'비트코인의 개발자 사토시'가 아니며, 동명이인임을 주장하였다. 또한 한때는 일본의 수학자로 교토 대학의 교수인 모치즈키 신이치가 사토시라고 지목되기도 했으나, 모치즈키 신이치는 자신은 사토시가 아니라고 했다. 실제로 사토시는 일본어를 쓴 적이 단 한번도 없었고, 비트코인 자체가 오픈 소스 소프트웨어 상에 구현된 전자화폐라서 실제 개발자가 누구인지 알아내기는 매우 힘드므로, 일본인이 아닐 가능성도 제기되었다. 또는'사토시 나카모토'라는 이름 자체가 동북아시아와 북아메리카 지역의 대표적인 대기업들인 나카미치, 모토로라, 삼성, 도시바의 4개 기업이 공동으로 쓰는 가명이라는 설도 제기된 바 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

## Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [7]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(3532812018032770127)
torch.cuda.manual_seed(3532812018032770127)
np.random.seed(324)
random.seed(2021)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [8]:
# Use subset (10 example) of original training dataset 
sample_idx = np.random.choice(range(len(dataset['train'])), 20)
training_dataset = dataset['train'][sample_idx]

Negative sampling을 위한 negative sample들을 샘플링

In [9]:
# set number of neagative sample
num_neg = 3

corpus = np.array(corpus)
p_with_neg = []

for c in training_dataset['context']:
  while True:
    neg_idxs = np.random.randint(len(corpus), size=num_neg)

    if not c in corpus[neg_idxs]:
      p_neg = corpus[neg_idxs]

      p_with_neg.append(c)
      p_with_neg.extend(p_neg)
      break


In [10]:
print('[Positive context]')
print(p_with_neg[0], '\n')
print('[Negative context]')
print(p_with_neg[1], '\n', p_with_neg[2])

[Positive context]
하지만 지속적으로 문제되었던 이른바 '줬다 뺏는 기초연금' 문제는 여전히 시정되지 못했다. 이는 노인 단독가구의 소득이 하위 70%에 속하지만 생계급여를 받는 경우에는 그 차이만큼 삭감되기 때문에 나온 말인데, 생계급여를 받지 못하는 차상위계층의 노인들만 사실상 기초연금을 받아가는 문제가 발생하는 것이다. 보건복지부는 생계급여를 받는 노인이 기초연금까지 받으면 기초생활보장 수급자에서 탈락한 노인들보다 오히려 소득이 높아질 수 있기 때문이라며, 현행 제도를 고수하겠다는 뜻을 나타냈지만 시민단체에서는 정부가 보충성의 원리를 너무 경직되게 적용한다고 주장하고 있다. 하지만 재정 부담이 크게 늘어날 것이라는 우려도 있다. 정부 계획대로 기초연금이 인상된다면 국민 1인당 추가 조세 부담액은 2018년 3만 원, 2030년 15만 원으로 급증할 것으로 분석됐다. 필요한 예산액도 내년 12조 7536억 원, 2030년 43조 6000억 원이 되어 마찬가지로 급증하게 된다. 

[Negative context]
서인영은 이 시기부터 본격적인 인기를 얻기 시작했다. MBC 예능 프로그램 《우리 결혼 했어요》에서 크라운 제이 와 호흡을 맞췄는데, 구두를 신상이라고 부르는 등 신조어를 만들어냈고 신상녀, 서인봉, 서인춘등 많은 별명이 붙으며 서인영의 이미지를 대중에게 각인시켜준 프로그램이였다. 비슷한 시기 M.net에서 방영된 리얼리티 프로그램 《서인영의 카이스트》에 출연하며 독특한 컨셉으로 많은 인기를 얻었다. 이 외에 많은 예능 프로그램에 출연하며 얼굴을 자주 비췄다. 《우리 결혼 했어요》에서 크라운 제이와 함께 부른 〈Too Much〉는 아무런 활동도 없었음에도 불과하고 1위를 차지했다. 2008년 7월 23일에는 싱글 〈Elly Is Cinderella〉를 발매하면서 두 번째 솔로 활동을 시작했다. 타이틀곡 〈신데렐라〉는 싸이의 작품으로 벨소리, 컬러링 차트 1위를 차지하는 등 인기를 얻었다. 과거 쥬얼리에서 존재감이 거의 없었던 서인영은

In [11]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors='pt')


In [12]:
max_len = p_seqs['input_ids'].size(-1)
p_seqs['input_ids'] = p_seqs['input_ids'].view(-1, num_neg+1, max_len)
p_seqs['attention_mask'] = p_seqs['attention_mask'].view(-1, num_neg+1, max_len)
p_seqs['token_type_ids'] = p_seqs['token_type_ids'].view(-1, num_neg+1, max_len)

print(p_seqs['input_ids'].size())  #(num_example, pos + neg, max_len)

torch.Size([20, 4, 512])


In [13]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러오기

In [14]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output


In [15]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.transf

Train function 정의 후, 두개의 encoder fine-tuning 하기


In [16]:
def train(args, num_neg, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

  # Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      targets = torch.zeros(args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
        targets = targets.cuda()

      p_inputs = {'input_ids': batch[0].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'attention_mask': batch[1].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'token_type_ids': batch[2].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1)
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}
      
      p_outputs = p_model(**p_inputs)  #(batch_size*(num_neg+1), emb_dim)
      q_outputs = q_model(**q_inputs)  #(batch_size*, emb_dim)

      # Calculate similarity score & loss
      p_outputs = p_outputs.view(args.per_device_train_batch_size, -1, num_neg+1)
      q_outputs = q_outputs.view(args.per_device_train_batch_size, 1, -1)

      sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()  #(batch_size, num_neg+1)
      sim_scores = sim_scores.view(args.per_device_train_batch_size, -1)
      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)
      print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()


    
  return p_model, q_model




In [17]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01
)


In [ ]:
p_encoder, q_encoder = train(args, num_neg, train_dataset, p_encoder, q_encoder)

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:  10%|█         | 1/10 [00:01<00:10,  1.20s/it]

tensor(1.1921e-07, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:  20%|██        | 2/10 [00:02<00:09,  1.21s/it]

...

tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:  10%|█         | 1/10 [00:01<00:11,  1.25s/it]

tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:  20%|██        | 2/10 [00:02<00:09,  1.25s/it]

tensor(0., device='cuda:0', grad_fn=<NllLossBackward>)


## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [19]:

valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth, '\n\n')

# valid_corpus

유아인에게 타고난 배우라고 말한 드라마 밀회의 감독은?
화보 촬영을 위해 미국에 있을 때, 김희애의 연락을 통해 JTBC 드라마 《밀회》의 캐스팅을 제안받았다. 당시 영화 《베테랑》에 이미 캐스팅된 상태였으나, 유아인은 류승완 감독과 제작사의 양해를 얻어 《밀회》에 출연한다. 천재 피아니스트 ‘이선재’ 역할을 위해 피아니스트들의 영상을 보고 곡의 스피드와 건반 위치 등을 외워 실제 타건을 하며 촬영했다. 피아노 울림판을 수건으로 막고 타건을 하면, 그 후 대역 피아니스트의 소리를 덧입히는 방식이었다. 《밀회》는 작품성을 인정받고 숱한 화제를 낳으며 당시 종편으로서는 높은 시청률을 기록했다. 유아인은 섬세한 연기력을 선보여 순수함으로 시청자들을 매료시켰다는 호평을 얻었고, 특히 피아노 연주에 있어서 클래식 종사자들에게 인정을 받았다. 연출을 맡은 안판석 감독은 유아인에 대해 “느낌으로만 연기를 하는 게 아니고 감성을 지적으로 통제해 가면서 연기한다. 그 나이에”라며 “타고난 배우”라고 말했다. 유아인은 《밀회》를 통해 예술적인 면모를 구체화할 수 있어서 만족감을 느꼈다고 밝혔으며, 종영 후 자신의 페이스북 계정에 긴 소감글을 남겼다. 특히 ‘이선재’ 캐릭터를 배우 유아인이 가진 소년성의 엑기스로 생각하며, 2015년 10월 부산국제영화제 오픈토크에서는 본인이 가장 좋아하는 캐릭터로 꼽았다. 




앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

In [20]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [21]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([11, 768]) torch.Size([1, 768])


생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

In [22]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

torch.Size([1, 11])
tensor([[ 8.5446,  6.7956,  4.5435,  6.3944,  6.1924, 13.5330,  6.9598, 11.5030,
          5.8191, 11.0207,  6.9937]])
tensor([ 5,  7,  9,  0, 10,  6,  1,  3,  4,  8,  2])


Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [23]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 유아인에게 타고난 배우라고 말한 드라마 밀회의 감독은? 

[Ground truth passage]
화보 촬영을 위해 미국에 있을 때, 김희애의 연락을 통해 JTBC 드라마 《밀회》의 캐스팅을 제안받았다. 당시 영화 《베테랑》에 이미 캐스팅된 상태였으나, 유아인은 류승완 감독과 제작사의 양해를 얻어 《밀회》에 출연한다. 천재 피아니스트 ‘이선재’ 역할을 위해 피아니스트들의 영상을 보고 곡의 스피드와 건반 위치 등을 외워 실제 타건을 하며 촬영했다. 피아노 울림판을 수건으로 막고 타건을 하면, 그 후 대역 피아니스트의 소리를 덧입히는 방식이었다. 《밀회》는 작품성을 인정받고 숱한 화제를 낳으며 당시 종편으로서는 높은 시청률을 기록했다. 유아인은 섬세한 연기력을 선보여 순수함으로 시청자들을 매료시켰다는 호평을 얻었고, 특히 피아노 연주에 있어서 클래식 종사자들에게 인정을 받았다. 연출을 맡은 안판석 감독은 유아인에 대해 “느낌으로만 연기를 하는 게 아니고 감성을 지적으로 통제해 가면서 연기한다. 그 나이에”라며 “타고난 배우”라고 말했다. 유아인은 《밀회》를 통해 예술적인 면모를 구체화할 수 있어서 만족감을 느꼈다고 밝혔으며, 종영 후 자신의 페이스북 계정에 긴 소감글을 남겼다. 특히 ‘이선재’ 캐릭터를 배우 유아인이 가진 소년성의 엑기스로 생각하며, 2015년 10월 부산국제영화제 오픈토크에서는 본인이 가장 좋아하는 캐릭터로 꼽았다. 

Top-1 passage with score 13.5330
김희선은 드라마 《스마일 어게인》 이후, 결혼과 출산으로 연기 활동을 중단한 이후, 2010년 중국 영화 《전국: 천하영웅의 시대》에 특별 출연하며 활동을 복귀하였다. 2012년에는 퓨전 사극 드라마 《신의》로 6년 만에 브라운관에 복귀하였다. 당초 그녀는 2년 전부터 이 작품의 시놉시스를 받고 출연이 정해져 있었지만 상대 남자배우의 캐스팅 난항으로 제작이 지연되면서 복귀도 예정보다 다소 늦어졌다. 이듬해, 2013년에는 《화신 - 마음을 지